# test of functions

uncomment and install these the first time you run this
TODO: needs to be in requirements

In [30]:
#!pip install geopy
#!pip install timezonefinder

### params.py

In [31]:
import os
import numpy as np

##### VARIABLES #####
METEO_VARIABLES = ["temperature_2m",
                   "relative_humidity_2m",
                   "dew_point_2m",
                   "apparent_temperature",
                   "precipitation",
                   "wind_speed_10m",
                   "cloud_cover",
                   "pressure_msl"]
RAW_DATA_PATH = os.path.join(os.path.expanduser('~'), "code", "pawarsp", "what-to-wear-today", "raw_data")


### utils.py

In [32]:
from geopy.geocoders import Nominatim
from timezonefinder import TimezoneFinder


def get_coords_from_location_name(location: str = "Berlin, Germany"):
    '''
    Translates place name like "Berlin, Germany" into (latitude, longitude)
    '''
    geolocator = Nominatim(user_agent="my_geocoder")
    location = geolocator.geocode(location)
    if location is None:
        print("Location could not be resolved. Using 'Berlin, Germany' instead.")
        location = geolocator.geocode("Berlin, Germany")
    return (location.latitude, location.longitude)

def get_timezone_from_coords(latitude, longitude):
    tf = TimezoneFinder()
    tz_name = tf.timezone_at(lat=latitude, lng=longitude)
    return tz_name


### data.py

In [33]:
import numpy as np
import pandas as pd
import openmeteo_requests
import requests_cache
from retry_requests import retry
#from .utils import get_coords_from_location_name, get_timezone_from_coords
#from .params import *

def get_meteo_data_with_cache(start_date: str = "2023-01-01",
                        end_date: str = "2025-10-15",
                        location: str = "Berlin, Germany"):
    """
    Retrieve data through the Open-Meteo API (), or from local file, if the file exists
    Stores data at raw_data/weatherdata_startdate_enddate_location.csv if retrieved from API for future use
    Returns a pandas.DataFrame
    """

    latitude, longitude = get_coords_from_location_name(location)
    timezone = get_timezone_from_coords(latitude, longitude)
    filepath = Path(RAW_DATA_PATH).joinpath(f'{location.replace(", ", "_").lower()}_{start_date.replace("-", "")}_{end_date.replace("-", "")}.csv')

    if filepath.is_file():
        print("Load meteo data from local CSV.")
        df = pd.read_csv(filepath, header="infer")
        if ((len(METEO_VARIABLES) + 1) != df.shape[1]) or not all([x in df.columns for x in METEO_VARIABLES]):
            print("Variables from local file are different from desired variables. Deleting local data.")
            os.remove(filepath)
            return get_meteo_data_with_cache(start_date, end_date, location)
    else:
        print("Load meteo data from Open-Meteo server.")

        # Setup the Open-Meteo API client with cache and retry on error
        cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
        retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
        openmeteo = openmeteo_requests.Client(session = retry_session)

        url = "https://archive-api.open-meteo.com/v1/archive"
        params = {
            "latitude": latitude,
            "longitude": longitude,
            "start_date": start_date,
            "end_date": end_date,
            "hourly": METEO_VARIABLES,
            "timezone": timezone,
        }
        responses = openmeteo.weather_api(url, params=params)
        response = responses[0]
        hourly = response.Hourly()
        data = {"date": pd.date_range(
            start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
            end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
            freq = pd.Timedelta(seconds = hourly.Interval()),
            inclusive = "left"
        )}

        for idx, var in enumerate(METEO_VARIABLES):
            data[var] = hourly.Variables(idx).ValuesAsNumpy()
        df = pd.DataFrame(data)

        # save data to csv
        if df.shape[0] > 1:
            df.to_csv(filepath, index=False)
    return df


In [35]:
start_date = "2023-01-01"
end_date = "2025-10-15"
location = "Berlin, Germany"

df = get_meteo_data_with_cache(start_date, end_date, location)
df.head()

Load meteo data from local CSV.
Variables from local file are different from desired variables. Deleting local data.
Load meteo data from Open-Meteo server.


date  temperature_2m  relative_humidity_2m  \
0 2022-12-31 23:00:00+00:00       14.571500             63.927788   
1 2023-01-01 00:00:00+00:00       15.321500             58.672153   
2 2023-01-01 01:00:00+00:00       15.721499             54.695515   
3 2023-01-01 02:00:00+00:00       15.771500             52.494183   
4 2023-01-01 03:00:00+00:00       15.871500             50.561550   

   dew_point_2m  apparent_temperature  precipitation  wind_speed_10m  \
0        7.8215              9.875454            0.0       28.312117   
1        7.2715             10.165958            0.0       30.532158   
2        6.6215             10.386935            0.0       30.699368   
3        6.0715             10.229070            0.0       31.259941   
4        5.6215             10.646867            0.0       28.394392   

   cloud_cover  pressure_msl  
0        100.0   1011.000000  
1        100.0   1010.799988  
2        100.0   1010.900024  
3        100.0   1011.299988  
4        100.0   1011.299988

# weather to sentence function

In [4]:
from datetime import datetime, timedelta
import pandas as pd

input = {"time":  [datetime.now() + timedelta(hours=i) for i in range(6)],
         "humidity": [60, 50, 50, 50, 60, 70],
         "temperature": [11, 11, 12, 12, 13, 13],
         "wind": [15, 16, 18, 20, 22, 30],
         "rain": [0, 0, 0, 0, 1.0, 1.5]}

df = pd.DataFrame(input)


def weather_sentence(row):
    time = row['time'].hour
    hum = row['humidity']
    if hum < 30:
        humid = "extremely dry"
    elif hum < 40:
        humid = "rather dry"
    elif hum < 60:
        humid = "neither dry nor humid"
    elif hum < 70:
        humid = "humid"
    elif hum < 85:
        humid = "pretty humid"
    else:
        humid = "moist"


    temp = row['temperature']
    if temp < -10:
        warmth = "insufferably cold, way below freezing"
    elif temp < 0:
        warmth = "below freezing"
    elif temp < 10:
        warmth = "rather cold and uncomfortable"
    elif temp < 15:
        warmth = "cool and fresh"
    elif temp < 18:
        warmth = "fresh and slightly uncomfortable"
    elif temp < 22:
        warmth = "mild and pleasant"
    elif temp < 26:
        warmth = "warm"
    elif temp < 30:
        warmth = "very warm, almost hot"
    else:
        warmth = "hot and unpleasant"

    wind = row['wind']
    if wind == 0:
        windy = "perfectly windstill"
    elif wind < 5:
        windy = "very calm"
    elif wind < 10:
        windy = "slightly breezy"
    elif wind < 20:
        windy = "breezy and gusty"
    elif wind < 30:
        windy = "stormy and very gusty"
    else:
        windy = "the time of a storm"


    rain = row['rain']
    if rain == 0:
        raining = "not raining, the air is clear."
    elif rain < 1:
        raining = "slightly drizzling"
    elif rain < 5:
        raining = "raining"
    elif rain < 10:
        raining = "raining heavily"
    else:
        raining = "raining cats and dogs"


    sn = f"At around {time} o'clock the air is {warmth} at {temp}°C, it is {windy} with a windspeed of {wind}km/h. "
    sn += f"The air feels {humid} with {hum}% humidity, it is {raining}."

    return sn

for i in range(df.shape[0]):
    print(weather_sentence(df.iloc[i]))


At around 15 o'clock the air is cool and fresh at 11°C, it is breezy and gusty with a windspeed of 15km/h. The air feels humid with 60% humidity, it is not raining, the air is clear..
At around 16 o'clock the air is cool and fresh at 11°C, it is breezy and gusty with a windspeed of 16km/h. The air feels neither dry nor humid with 50% humidity, it is not raining, the air is clear..
At around 17 o'clock the air is cool and fresh at 12°C, it is breezy and gusty with a windspeed of 18km/h. The air feels neither dry nor humid with 50% humidity, it is not raining, the air is clear..
At around 18 o'clock the air is cool and fresh at 12°C, it is stormy and very gusty with a windspeed of 20km/h. The air feels neither dry nor humid with 50% humidity, it is not raining, the air is clear..
At around 19 o'clock the air is cool and fresh at 13°C, it is stormy and very gusty with a windspeed of 22km/h. The air feels humid with 60% humidity, it is raining.
At around 20 o'clock the air is cool and fres